In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%shell

#Download the compressed model file
echo "Downloading Dlib 68-point shape predictor..."
wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2

# Extract it
echo "Extracting model..."
bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2

# Confirm
if [ -f shape_predictor_68_face_landmarks.dat ]; then
    echo "✅ Model downloaded and extracted successfully!"
else
    echo "❌ Something went wrong."
fi

--2025-06-12 10:54:33--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 [following]
--2025-06-12 10:54:33--  https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Connecting to dlib.net (dlib.net)|107.180.26.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  33.7MB/s    in 1.8s    

2025-06-12 10:54:35 (33.7 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]

Extracting model...
✅ Model downloaded and extracted successfully!


In [ ]:
import cv2
import dlib
import os
from imutils import face_utils
from tqdm import tqdm

# Landmark model path
landmark_model = '/content/shape_predictor_68_face_landmarks.dat'

def create_occluded_regions_dataset(input_dir):

    # Define
    output_base_dir = input_dir.rstrip('/') + '-occluded-regions'
    last_dir_name = '/'.join(input_dir.rstrip('/').split(os.sep)[-2:])

    # Create output folders
    regions = ['left_eye', 'right_eye', 'nose', 'mouth']
    for region in regions:
        os.makedirs(os.path.join(output_base_dir, region), exist_ok=True)

    # Load Dlib models
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(landmark_model)

    # Region landmarks
    region_indices = {
        'left_eye': list(range(36, 42)),
        'right_eye': list(range(42, 48)),
        'nose': list(range(27, 36)),
        'mouth': list(range(48, 68)),
    }

    def occlude_region(image, landmarks, indices):
        pts = landmarks[indices]

        # Expand the occlusion region by a scale factor (e.g., 1.5 = 50% larger)
        scale = 1.5
        x, y, w, h = cv2.boundingRect(pts)

        # Compute center and new size
        cx, cy = x + w // 2, y + h // 2
        new_w, new_h = int(w * scale), int(h * scale)
        new_x = max(0, cx - new_w // 2)
        new_y = max(0, cy - new_h // 2)

        # Make sure we don't go out of image bounds
        new_x2 = min(image.shape[1], new_x + new_w)
        new_y2 = min(image.shape[0], new_y + new_h)

        # Apply the occlusion
        cv2.rectangle(image, (new_x, new_y), (new_x2, new_y2), (0, 0, 0), -1)

        return image

    # Collect all image files recursively
    image_files = []
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.lower().endswith(('.jpg', '.png', '.jpeg')):
                # Save full path and relative path for output
                full_path = os.path.join(root, file)
                rel_path = os.path.relpath(full_path, input_dir)
                image_files.append((full_path, rel_path))

    # Process images with progress bar
    for full_path, rel_path in tqdm(image_files, desc=f"Landmark occlusion: {last_dir_name}"):
        image = cv2.imread(full_path)
        if image is None:
            print(f"Warning: Failed to load {full_path}. Skipping.")
            continue

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)

        if len(faces) == 0:
            print(f"No face detected in {rel_path}. Skipping.")
            continue

        shape = predictor(gray, faces[0])
        landmarks = face_utils.shape_to_np(shape)

        for region, indices in region_indices.items():
            img_copy = image.copy()
            occluded = occlude_region(img_copy, landmarks, indices)

            # Preserve subdirectory structure in output
            output_path = os.path.join(output_base_dir, region, rel_path)
            output_dir = os.path.dirname(output_path)
            os.makedirs(output_dir, exist_ok=True)

            cv2.imwrite(output_path, occluded)

    print("✅ Occlusion completed!")


In [ ]:
import cv2
import os
from tqdm import tqdm


def create_grid_occlusion_dataset(input_dir, rows=4, cols=4):
    last_dir_name = '/'.join(input_dir.rstrip('/').split(os.sep)[-2:])

    def occlude_grid_cell(image, rows, cols, row_idx, col_idx):
        h, w = image.shape[:2]
        cell_h, cell_w = h // rows, w // cols

        # Calculate cell coordinates
        x1 = col_idx * cell_w
        y1 = row_idx * cell_h
        x2 = min(w, x1 + cell_w)
        y2 = min(h, y1 + cell_h)

        # Make a copy and apply occlusion
        occluded = image.copy()
        cv2.rectangle(occluded, (x1, y1), (x2, y2), (0, 0, 0), -1)
        return occluded

    output_base_dir = input_dir.rstrip('/') + f'-grid-{rows}x{cols}-occluded'
    os.makedirs(output_base_dir, exist_ok=True)

    # Collect all image files recursively
    image_files = []
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.lower().endswith(('.jpg', '.png', '.jpeg')):
                full_path = os.path.join(root, file)
                rel_path = os.path.relpath(full_path, input_dir)
                image_files.append((full_path, rel_path))

    # Process each image
    for full_path, rel_path in tqdm(image_files, desc=f"Grid occlusion: {last_dir_name}"):
        image = cv2.imread(full_path)
        if image is None:
            print(f"Warning: Failed to load {full_path}. Skipping.")
            continue

        for row in range(rows):
            for col in range(cols):
                occluded = occlude_grid_cell(image, rows, cols, row, col)

                # Save to respective directory
                grid_name = f"r{row}_c{col}"
                output_path = os.path.join(output_base_dir, grid_name, rel_path)
                os.makedirs(os.path.dirname(output_path), exist_ok=True)
                cv2.imwrite(output_path, occluded)

    print("✅ Grid-based occlusion completed!")


In [ ]:
dataset_to_process = [
    '/content/drive/MyDrive/mad/code/ViT-SVM/datasets/frll-cropped/bonafide/',
    '/content/drive/MyDrive/mad/code/ViT-SVM/datasets/frll-cropped/morphs/',
    '/content/drive/MyDrive/mad/code/ViT-SVM/datasets/ffhq-cropped/morphs/',
    ]

In [ ]:
for dataset in dataset_to_process:
    create_grid_occlusion_dataset(dataset)

Grid occlusion: frll-cropped/bonafide: 100%|██████████| 204/204 [03:07<00:00,  1.09it/s]


✅ Grid-based occlusion completed!


Grid occlusion: frll-cropped/morphs: 100%|██████████| 7061/7061 [1:27:10<00:00,  1.35it/s]


✅ Grid-based occlusion completed!


Grid occlusion: ffhq-cropped/morphs: 100%|██████████| 25012/25012 [7:35:48<00:00,  1.09s/it]

✅ Grid-based occlusion completed!


In [ ]:
for dataset in dataset_to_process:
    create_occluded_regions_dataset(dataset)

Landmark occlusion: frll-cropped/bonafide:  15%|█▍        | 30/204 [00:09<00:47,  3.68it/s]

No face detected in neutral/082_03.png. Skipping.


Landmark occlusion: frll-cropped/bonafide:  41%|████      | 83/204 [00:23<00:31,  3.78it/s]

No face detected in neutral/042_03.png. Skipping.


Landmark occlusion: frll-cropped/bonafide: 100%|██████████| 204/204 [01:35<00:00,  2.14it/s]


...

No face detected in stylegan-interp-W+/67247_53155.png. Skipping.


Landmark occlusion: ffhq-cropped/morphs: 100%|█████████▉| 24954/25012 [3:22:18<00:19,  2.96it/s]

No face detected in stylegan-interp-W+/36444_50210.png. Skipping.


Landmark occlusion: ffhq-cropped/morphs: 100%|██████████| 25012/25012 [3:22:42<00:00,  2.06it/s]

✅ Occlusion completed!


In [ ]:
%%shell
# Delete any files with 0 bytes (error while processing)
find /content/drive/MyDrive/mad/code/ViT-SVM/datasets/ -type f -size 0 -delete

In [ ]:
import time
from google.colab import runtime

#time.sleep(15 * 60)

runtime.unassign()